In [1]:
import pandas as pd
import numpy as np
import redefine

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, DBSCAN, HDBSCAN, AgglomerativeClustering
from sklearn.metrics import accuracy_score

In [2]:
from pprint import pprint

In [3]:
data = pd.read_csv('data/iris_true.csv')

In [4]:
data

,id,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,0,5.1,3.5,1.4,0.2,setosa
1,1,4.9,3.0,1.4,0.2,setosa
2,2,4.7,3.2,1.3,0.2,setosa
3,3,4.6,3.1,1.5,0.2,setosa
4,4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...,...
145,145,6.7,3.0,5.2,2.3,virginica
146,146,6.3,2.5,5.0,1.9,virginica
147,147,6.5,3.0,5.2,2.0,virginica
148,148,6.2,3.4,5.4,2.3,virginica


In [5]:
rd = redefine.REDEFINE('iris_true.csv', data, 'target', 'id')

In [6]:
results = rd.run_redefine('Random Forest', {'n_estimators':'5'}, 'KMeans', {}, 'Standard')

In [ ]:
results.to_csv('testing.txt', sep='\t')

In [ ]:
X = rd.get_X()
IDs = rd.get_IDs()
Y = rd.get_Y()
Y_names = rd.get_Y_names()
kfolds = 10

In [ ]:
def accuracy_score(ytest, yhat):
        nz = np.flatnonzero(ytest == yhat)
        return len(nz)/len(ytest)

def doKFold(model, scaler):
        n = len(X)
        idxs = np.linspace(0, n, kfolds+1).astype(int)
        idx = np.arange(0, n)
        
        np.random.shuffle(idx)

        x = X.copy()[idx]
        y = Y.copy()[idx]
        ids = IDs.copy()[idx]

        accuracy_scores = np.zeros(kfolds)
        
        ####
        idtests = np.zeros(n, dtype='O')
        yhattests = np.zeros(n, dtype='O')
        ####

        for k, i in enumerate(range(1, len(idxs))):
            idx1 = idxs[i-1]
            idx2 = idxs[i]

            xtest = x[idx1:idx2]
            xtrain = np.concatenate([x[idxs[0]:idx1], x[idx2:idxs[-1]]])

            ytest = y[idx1:idx2]
            ytrain = np.concatenate([y[idxs[0]:idx1], y[idx2:idxs[-1]]])

            if scaler is not None:
                xtrain = scaler.fit_transform(xtrain)
                xtest = scaler.transform(xtest)

            model.fit(xtrain, ytrain)
            yhat = model.predict(xtest)
            
            accuracy_scores[k] = accuracy_score(ytest, yhat)
            
            ####
            idtests[idx1:idx2] = ids[idx1:idx2]
            yhattests[idx1:idx2] = yhat
            ####
        #, ####
        return np.mean(accuracy_scores), zip(idtests, yhattests)

In [ ]:
testarr = np.zeros(10, dtype='O')
testarr[0:3] = np.array(['1', '1', '1'])
testarr

array(['1', '1', '1', 0, 0, 0, 0, 0, 0, 0], dtype=object)

In [ ]:
model = RandomForestClassifier()
scaler = StandardScaler()
score, results = doKFold(model, scaler)

In [ ]:
results

In [ ]:
score

0.9572463768115942

In [ ]:
type(IDs)

pandas.core.series.Series

In [ ]:
df = pd.DataFrame(columns=['Label','ClassificationResult', 'ClusterResult'], index=IDs)

In [ ]:
df

,Label,ClassificationResult,ClusterResult
id,,,
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
145,NaN,NaN,NaN
146,NaN,NaN,NaN
147,NaN,NaN,NaN


In [ ]:
row0 = df.loc[0]
row0

Label                   NaN
ClassificationResult    NaN
ClusterResult           NaN
Name: 0, dtype: object

In [ ]:
row0['ClassificationResult'] == row0['ClusterResult']

False

In [ ]:
df = pd.read_csv('output/results_iris_true_2023-11-10_00-48-51.703759.csv', index_col='id')
df

,Label,ClassificationResult,ClusterResult
id,,,
0,setosa,setosa,setosa
1,setosa,setosa,setosa
2,setosa,setosa,setosa
3,setosa,setosa,setosa
4,setosa,setosa,setosa
...,...,...,...
145,virginica,virginica,virginica
146,virginica,virginica,versicolor
147,virginica,virginica,virginica


In [ ]:
for index, row in df.iterrows():
    if (row['ClassificationResult'] == row['ClusterResult']) and \
       (row['Label'] != row['ClassificationResult']):
        print(index)
        df.at[index, 'Flagged'] = True

70
77
106
119
133


/var/folders/t9/rcsp8s0s077g3tn69frcn8lw0000gn/T/ipykernel_53571/3658539165.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Flagged'] = True


In [ ]:
df

,Label,ClassificationResult,ClusterResult,Flagged
id,,,,
0,setosa,setosa,setosa,NaN
1,setosa,setosa,setosa,NaN
2,setosa,setosa,setosa,NaN
3,setosa,setosa,setosa,NaN
4,setosa,setosa,setosa,NaN
...,...,...,...,...
145,virginica,virginica,virginica,NaN
146,virginica,virginica,versicolor,NaN
147,virginica,virginica,virginica,NaN


In [ ]:
df.to_csv('testing.csv')

In [8]:
t = ['a','b','c','d']

In [10]:
tt = str(t)

In [11]:
tt.replace('[','').replace("'",'').replace(']','')

'a, b, c, d'